In [61]:
import pandas as pd
import os
os.chdir("/home/yangkai/codegen-detection/")
# hum_data = pd.read_pickle("dataset/mbpp_hum_test.pkl")
gen_data = pd.read_pickle("dataset/apps_gen_test100_gpt4.pkl")

In [70]:
import re
from prepro.code_prepro import CommentsRemover,PygmentsTokenizer,EmptyLinesRemover

def extract_code(resp,id=0):
    try:
        text = resp['choices'][id]['message']['content']
        # text = resp[1][0]['generated_text']
    except KeyError:
        return ''
    pattern = r'```python\n(.+?)\n```'
    code_blocks = re.findall(pattern, text, re.DOTALL)
    if len(code_blocks) > 0:
        return code_blocks[0]

    if resp['choices'][id]['finish_reason'] == 'length':
        pattern2 = r'```python\n(.+?)'
    else:
        pattern2 = r'``` python\n(.+?)\n```'

    # pattern2 = r'```python\n(.+?)'
    code_blocks = re.findall(pattern2, text, re.DOTALL)
    if len(code_blocks) > 0:
        return code_blocks[0]

    pattern3 = r"```\n(.+?)\n```"
    code_blocks = re.findall(pattern3, text, re.DOTALL)
    if len(code_blocks) > 0:
        return code_blocks[0]

    if resp['choices'][id]['finish_reason'] == 'length':
        pattern4 = r"```\n(.+?)"
        code_blocks = re.findall(pattern4, text, re.DOTALL)
        if len(code_blocks) > 0:
            return code_blocks[0]

    # pattern4 = r"```\n(.+?)"
    # code_blocks = re.findall(pattern4, text, re.DOTALL)
    # if len(code_blocks) > 0:
    #     return code_blocks[0]

    return ""

import json
def load_json_dataset(pth):
    dataset = []
    cnt = 0
    with open(pth,"r") as f:
        for line in f:
            try:
                dataset.append(json.loads(line.strip()))
                cnt += 1
            except:
                dataset.append([{},{}])

    return dataset

idx = 3
gen_resp = load_json_dataset("prompt_gpt/apps_gen_test100_gpt35_temp20_gpt35_resample.jsonl")
extract_gen = [extract_code(resp,id=idx) for resp in gen_resp]

tokenizer = PygmentsTokenizer('Python')
comment_remover = CommentsRemover(tokenizer)
emptyline_remover = EmptyLinesRemover()

def prepro_pipeline(x):
    return emptyline_remover.filter(comment_remover.filter(x))

extract_gen = [prepro_pipeline(x) for x in extract_gen]
len(list(filter(lambda x:len(x)>0,extract_gen)))

99

In [71]:
gen_data[f"gpt_resample_code_cleaned{idx+1}"] = extract_gen
# hum_data['extracted_full_func'] = hum_data['solution']

/tmp/ipykernel_2396855/1903384213.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  gen_data[f"gpt_resample_code_cleaned{idx+1}"] = extract_gen


In [48]:
# gen_data['extracted_full_func'] = gen_data['extracted_full_func'].apply(prepro_pipeline)
# hum_data['extracted_full_func'] = hum_data['extracted_full_func'].apply(prepro_pipeline)
gen_data['extracted_full_func'] = extract_gen

In [72]:
gen_data.to_pickle("dataset/apps_gen_test100_gpt35_temp20.pkl")
# hum_data.to_pickle("dataset/mbpp_hum_test.pkl")

In [11]:
gen_data['extracted_full_func'].tolist()[:10]

["def encode(message, key, initShift):\n    unique_key = ''.join(sorted(set(key), key=key.index))\n    rest_of_alpha = ''.join(sorted(set('abcdefghijklmnopqrstuvwxyz') - set(unique_key)))\n    key_alphabet = unique_key + rest_of_alpha\n    result = ''\n    for char in message:\n        if char.isalpha():\n            index = (key_alphabet.index(char) + initShift) % 26\n            result += key_alphabet[index]\n            initShift = key_alphabet.index(char) + 1\n        else:\n            result += char\n    return result\ndef decode(message, key, initShift):\n    unique_key = ''.join(sorted(set(key), key=key.index))\n    rest_of_alpha = ''.join(sorted(set('abcdefghijklmnopqrstuvwxyz') - set(unique_key)))\n    key_alphabet = unique_key + rest_of_alpha\n    result = ''\n    for char in message:\n        if char.isalpha():\n            index = (key_alphabet.index(char) - initShift) % 26\n            result += key_alphabet[index]\n            initShift = key_alphabet.index(result[-1]) +